In [2]:
import pandas as pd
import numpy as np
import gzip
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re

Carga de Datos

In [3]:
steam_games = pd.read_json('../Data/steam_games.json.gz',compression='gzip',lines=True)

In [4]:
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [5]:
#Verificación de datos nulos

steam_games.isna().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [6]:
#Se eliminan las filas que cuyos datos eran completamente Nulas

steam_games = steam_games.dropna(how='all')

In [7]:
#Se vuelve a verificar cuántos datos nulos quedan en el Dataframe
steam_games.isna().sum()


publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [8]:
#Examinamos los datos nulos de la columna app_name.

steam_games[steam_games.app_name.isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
90890,None,"[Action, Indie]",None,None,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",None,0.0,317160.0,None


In [9]:
#Al constatar que dichos registros no poseen app_name, publisher, title y developer, se procede a eliminarlos.
steam_games.dropna(subset=['app_name'], inplace=True)

C:\Users\belen\AppData\Local\Temp\ipykernel_720\2891395286.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games.dropna(subset=['app_name'], inplace=True)


In [10]:
steam_games.isna().sum()

publisher       8050
genres          3282
app_name           0
title           2048
url                0
release_date    2066
tags             162
reviews_url        1
specs            669
price           1376
early_access       0
id                 1
developer       3297
dtype: int64

In [11]:
#Se observó que las columnas app_name y title poseen información en común, procediéndose a completar los nulos 
#de la columna title con la información de la columna app_name
steam_games['title'] = steam_games['app_name'].fillna(steam_games['title'])

C:\Users\belen\AppData\Local\Temp\ipykernel_720\6196267.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['title'] = steam_games['app_name'].fillna(steam_games['title'])


In [12]:
#Se normalizan los nombres de los registros que se encuentran en la columna app_name

steam_games['app_name'] = steam_games['app_name'].str.capitalize()

C:\Users\belen\AppData\Local\Temp\ipykernel_720\420666779.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['app_name'] = steam_games['app_name'].str.capitalize()


In [13]:
steam_games.isna().sum()

publisher       8050
genres          3282
app_name           0
title              0
url                0
release_date    2066
tags             162
reviews_url        1
specs            669
price           1376
early_access       0
id                 1
developer       3297
dtype: int64

In [14]:
#Comprobamos que las columnas genre y tags comparten información en común. Por lo que se procedió a completar 
#los registros vacios de genres con la información que se encuentra en tags

steam_games['genres'] = steam_games['tags'].fillna(steam_games['genres'])

C:\Users\belen\AppData\Local\Temp\ipykernel_720\147899129.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['genres'] = steam_games['tags'].fillna(steam_games['genres'])


In [15]:
# Procedemos a hacer lo mismo con la columna tags
steam_games['tags'] = steam_games['genres'].fillna(steam_games['tags'])


C:\Users\belen\AppData\Local\Temp\ipykernel_720\2164239591.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['tags'] = steam_games['genres'].fillna(steam_games['tags'])


In [16]:
steam_games.isna().sum()

publisher       8050
genres           138
app_name           0
title              0
url                0
release_date    2066
tags             138
reviews_url        1
specs            669
price           1376
early_access       0
id                 1
developer       3297
dtype: int64

In [17]:
# Se procedió a verificar los nulos que poseían las columnas tags y specs, encontrándose que dichos registros
# tampoco poseen publisher y developer.
steam_games[steam_games['tags'].isna() & steam_games['specs'].isna()]


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
93683,None,None,Brainbread 2 mod tools,BrainBread 2 Mod Tools,http://store.steampowered.com/app/382990/Brain...,2016-02-17,None,http://steamcommunity.com/app/382990/reviews/?...,None,None,0.0,382990.0,None
97870,None,None,Alien swarm: reactive drop dedicated server,Alien Swarm: Reactive Drop Dedicated Server,http://store.steampowered.com/app/582400/Alien...,2017-04-20,None,http://steamcommunity.com/app/582400/reviews/?...,None,None,0.0,582400.0,None
109645,None,None,Steam controller skin - csgo blue/orange,Steam Controller Skin - CSGO Blue/Orange,http://store.steampowered.com/app/531340/Steam...,2016-09-29,None,http://steamcommunity.com/app/531340/reviews/?...,None,9.99,0.0,531340.0,None
109646,None,None,Steam controller skin - csgo grey camo,Steam Controller Skin - CSGO Grey Camo,http://store.steampowered.com/app/531360/Steam...,2016-09-29,None,http://steamcommunity.com/app/531360/reviews/?...,None,9.99,0.0,531360.0,None
109647,None,None,Steam link skin - csgo blue/orange,Steam Link Skin - CSGO Blue/Orange,http://store.steampowered.com/app/531430/Steam...,2016-09-29,None,http://steamcommunity.com/app/531430/reviews/?...,None,9.99,0.0,531430.0,None
111636,None,None,Is defense editor,IS Defense Editor,http://store.steampowered.com/app/451330/IS_De...,2016-04-29,None,http://steamcommunity.com/app/451330/reviews/?...,None,None,0.0,451330.0,None
116048,None,None,Bomb dedicated server,BOMB Dedicated Server,http://store.steampowered.com/app/338440/BOMB_...,2014-12-17,None,http://steamcommunity.com/app/338440/reviews/?...,None,None,0.0,338440.0,None


In [18]:
# Se procede a eliminar dichos registros 

eliminar_nulos= steam_games[steam_games['tags'].isna() & steam_games['specs'].isna()].index
steam_games.drop(index=eliminar_nulos, inplace=True)

C:\Users\belen\AppData\Local\Temp\ipykernel_720\2916812363.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games.drop(index=eliminar_nulos, inplace=True)


In [19]:
steam_games.isna().sum()

publisher       8043
genres           131
app_name           0
title              0
url                0
release_date    2066
tags             131
reviews_url        1
specs            662
price           1372
early_access       0
id                 1
developer       3290
dtype: int64

In [20]:
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Strategy, Action, Indie, Casual, Simulation]",Lost summoner kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Strategy, Indie, RPG, Card Game...",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Free to Play, Simulation, Sports, Casual, Ind...",Real pool 3d - poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,"[Action, Indie, Casual, Sports]",Log challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Strategy, Indie, Casual, Simulation]",Colony on mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Strategy, Indie, Casual]",Logistical: south africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Simulation, Racing]",Russian roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",Exit 2 - directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [21]:
#Se encontró que los Id aparecen en las url, procediéndose a completar el registro nulo de la columna id con ese valor 
steam_games[steam_games['id'].isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Open World, Batman, Adventure, Stealt...",Batman: arkham city - game of the year edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [22]:
url = steam_games[steam_games['id'].isna()]['url'].values[0]

def input_id (url):
    # Se define la expresión regular
    expresion_regular = r"/app/(\d+)"

    #Buscamos el número en la URL
    resultado = re.search(expresion_regular, url)

    #Si encuentra el patrón, extraemos el número
    if resultado:
        numero = resultado.group(1)
        return int(numero)
    else:
        return None 

In [23]:
#Aplicamos la función en la columna 'url'

steam_games['url'].apply(input_id)

88310     761140
88311     643980
88312     670290
88313     767400
88314     773570
           ...  
120440    773640
120441    733530
120442    610660
120443    658870
120444    681550
Name: url, Length: 32126, dtype: int64

In [24]:
#Guardamos el dato en la columna 'id'

steam_games.loc[steam_games['id'].isna(), 'id'] = steam_games.loc[steam_games['id'].isna(), 'url'].apply(input_id)

In [25]:
#Se encontró que existen columnas duplicadas 

steam_games[steam_games['id'] == 200260]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
89378,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Open World, Batman, Adventure, Stealt...",Batman: arkham city - game of the year edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,200260.0,"Rocksteady Studios,Feral Interactive (Mac)"
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Open World, Batman, Adventure, Stealt...",Batman: arkham city - game of the year edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,200260.0,"Rocksteady Studios,Feral Interactive (Mac)"


In [26]:
steam_games['id'].value_counts()

id
200260.0    2
612880.0    2
530200.0    1
518690.0    1
513460.0    1
           ..
430094.0    1
676060.0    1
494160.0    1
215280.0    1
681550.0    1
Name: count, Length: 32124, dtype: int64

In [27]:
#Se procedió a eliminar las columnas duplicadas

steam_games.drop_duplicates(subset=['id'], inplace=True)

C:\Users\belen\AppData\Local\Temp\ipykernel_720\3425683958.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games.drop_duplicates(subset=['id'], inplace=True)


In [28]:
steam_games['id'].value_counts()

id
761140.0    1
534260.0    1
418670.0    1
462680.0    1
518690.0    1
           ..
676060.0    1
494160.0    1
215280.0    1
667090.0    1
681550.0    1
Name: count, Length: 32124, dtype: int64

In [29]:
steam_games.isna().sum()

publisher       8043
genres           131
app_name           0
title              0
url                0
release_date    2066
tags             131
reviews_url        0
specs            662
price           1372
early_access       0
id                 0
developer       3290
dtype: int64

In [30]:
# Se comprobó que las columnas publisher y developer poseen registros en común, por lo que se procedió
#a rellenarlas entre si 
steam_games['developer'] = steam_games['publisher'].fillna(steam_games['developer'])

C:\Users\belen\AppData\Local\Temp\ipykernel_720\1391811959.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['developer'] = steam_games['publisher'].fillna(steam_games['developer'])


In [31]:
steam_games['publisher'] = steam_games['developer'].fillna(steam_games['publisher'])

C:\Users\belen\AppData\Local\Temp\ipykernel_720\3462456357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['publisher'] = steam_games['developer'].fillna(steam_games['publisher'])


In [32]:
#Ponemos categoria Otros en publisher y developer que tengan valores nulos 
steam_games['publisher'].fillna('otros', inplace=True)
steam_games['developer'].fillna('otros', inplace=True)

C:\Users\belen\AppData\Local\Temp\ipykernel_720\2722531942.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['publisher'].fillna('otros', inplace=True)
C:\Users\belen\AppData\Local\Temp\ipykernel_720\2722531942.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['developer'].fillna('otros', inplace=True)


In [33]:
steam_games['publisher']

88310            Kotoshiro
88311     Making Fun, Inc.
88312         Poolians.com
88313                 彼岸领域
88314                otros
                ...       
120440     Ghost_RUS Games
120441              Sacada
120442        Laush Studio
120443            SIXNAILS
120444               otros
Name: publisher, Length: 32124, dtype: object

In [34]:
#Se normalizan los nombres de los registros que se encuentran en las columnas publisher y developer

steam_games['developer'] = steam_games['developer'].str.capitalize()
steam_games['publisher'] = steam_games['publisher'].str.capitalize()


C:\Users\belen\AppData\Local\Temp\ipykernel_720\1746888469.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['developer'] = steam_games['developer'].str.capitalize()
C:\Users\belen\AppData\Local\Temp\ipykernel_720\1746888469.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['publisher'] = steam_games['publisher'].str.capitalize()


In [35]:
steam_games['publisher']

88310            Kotoshiro
88311     Making fun, inc.
88312         Poolians.com
88313                 彼岸领域
88314                Otros
                ...       
120440     Ghost_rus games
120441              Sacada
120442        Laush studio
120443            Sixnails
120444               Otros
Name: publisher, Length: 32124, dtype: object

In [36]:
# Eliminamos espacios en blanco
steam_games[['developer' , 'publisher']] = steam_games[['developer', 'publisher']].apply(lambda x: x.str.strip())

C:\Users\belen\AppData\Local\Temp\ipykernel_720\516718044.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games[['developer' , 'publisher']] = steam_games[['developer', 'publisher']].apply(lambda x: x.str.strip())


In [37]:
steam_games.isna().sum()

publisher          0
genres           131
app_name           0
title              0
url                0
release_date    2066
tags             131
reviews_url        0
specs            662
price           1372
early_access       0
id                 0
developer          0
dtype: int64

In [38]:
# En la columna Price filtramos con expresión regular todos los registros que no sean datos numéricos

patron = r'\D+'
resultados = steam_games[steam_games['price'].str.contains(patron, case=False, na=False)]

resultados['price'].unique()

array(['Free To Play', 'Free to Play', 'Free', 'Free Demo',
       'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo',
       'Free Mod', 'Install Theme', 'Third-party', 'Play Now',
       'Free HITMAN™ Holiday Pack', 'Play the Demo',
       'Starting at $499.00', 'Starting at $449.00', 'Free to Try',
       'Free Movie', 'Free to Use'], dtype=object)

In [39]:
# Luego creamos un diccionario con los nombres de dichos registros y  les imputamos un precio.

dicc_price = {'Free To Play': 0, 
              'Free to Play': 0, 
              'Free': 0, 
              'Free Demo': 0,
              'Play for Free!': 0,
              'Install Now': 0, 
              'Play WARMACHINE: Tactics Demo': 0,
              'Free Mod': 0, 
              'Install Theme': 0, 
              'Third-party': 0,
              'Play Now': 0,
              'Free HITMAN™ Holiday Pack': 0, 
              'Play the Demo': 0,
              'Starting at $499.00': 499, 
              'Starting at $449.00': 449, 
              'Free to Try': 0,
              'Free Movie': 0, 
              'Free to Use': 0}

In [40]:
#Reemplazamos en la columna Price dichos registros con los valores imputados en el diccionario creado

steam_games['price'] = steam_games['price'].replace(dicc_price)

C:\Users\belen\AppData\Local\Temp\ipykernel_720\1377791351.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['price'] = steam_games['price'].replace(dicc_price)


In [41]:
#Imputamos los valores nulos de la columna price y early access

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp_mean = IterativeImputer(random_state=0)

steam_games[['price','early_access']] = imp_mean.fit_transform(steam_games[['price', 'early_access']])

C:\Users\belen\AppData\Local\Temp\ipykernel_720\197037167.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games[['price','early_access']] = imp_mean.fit_transform(steam_games[['price', 'early_access']])


In [42]:
steam_games.describe()

,price,early_access,id
count,32124.000000,32124.000000,3.212400e+04
mean,8.895646,0.060609,4.517508e+05
std,15.986925,0.238615,1.827313e+05
min,0.000000,0.000000,1.000000e+01
25%,2.990000,0.000000,3.292675e+05
50%,4.990000,0.000000,4.520300e+05
75%,9.990000,0.000000,5.934125e+05
max,995.000000,1.000000,2.028850e+06


In [43]:
steam_games.isna().sum()

publisher          0
genres           131
app_name           0
title              0
url                0
release_date    2066
tags             131
reviews_url        0
specs            662
price              0
early_access       0
id                 0
developer          0
dtype: int64

In [44]:
# Cambiamos a formato fecha la columna release_date y pusumos nulos los que no tenian un formato adecuado para poder 
# establecer una fecha

steam_games['release_date'] = pd.to_datetime(steam_games['release_date'], format='%Y-%m-%d', errors='coerce')

C:\Users\belen\AppData\Local\Temp\ipykernel_720\4293364593.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['release_date'] = pd.to_datetime(steam_games['release_date'], format='%Y-%m-%d', errors='coerce')


In [45]:
#Verificamos los valores nulos de la columna release_date

steam_games[steam_games['release_date'].isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88314,Otros,"[Action, Indie, Casual, Sports]",Log challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaT,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.990000,0.0,773570.0,Otros
88320,Qucheza,"[Early Access, Action, Indie, Simulation, Surv...",Uncanny islands,Uncanny Islands,http://store.steampowered.com/app/768570/Uncan...,NaT,"[Early Access, Action, Indie, Simulation, Surv...",http://steamcommunity.com/app/768570/reviews/?...,[Single-player],9.854708,1.0,768570.0,Qucheza
88321,Otros,[Casual],Icarus six sixty six,Icarus Six Sixty Six,http://store.steampowered.com/app/724910/Icaru...,NaT,[Casual],http://steamcommunity.com/app/724910/reviews/?...,"[Single-player, HTC Vive, Tracked Motion Contr...",0.000000,0.0,724910.0,Otros
88329,Otros,"[Early Access, Indie, VR]",After life vr,After Life VR,http://store.steampowered.com/app/772590/After...,NaT,"[Early Access, Indie, VR]",http://steamcommunity.com/app/772590/reviews/?...,"[Single-player, HTC Vive, Tracked Motion Contr...",4.990000,1.0,772590.0,Otros
88330,Otros,"[Early Access, Action, Adventure, Indie, Casual]",Kitty hawk,Kitty Hawk,http://store.steampowered.com/app/640250/Kitty...,NaT,"[Early Access, Action, Adventure, Indie, Casual]",http://steamcommunity.com/app/640250/reviews/?...,"[Single-player, Steam Leaderboards, HTC Vive, ...",2.990000,1.0,640250.0,Otros
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120397,"Greyson richey, nicholas lives","[Casual, Action, Indie, Simulation]",Lonely astronaut,Lonely Astronaut,http://store.steampowered.com/app/755830/Lonel...,NaT,"[Casual, Action, Indie, Simulation]",http://steamcommunity.com/app/755830/reviews/?...,[Single-player],8.833768,0.0,755830.0,"Greyson richey, nicholas lives"
120398,Rechargecomplete,"[Early Access, Action, Adventure, Indie]",Recharge complete,RECHARGE COMPLETE,http://store.steampowered.com/app/708070/RECHA...,NaT,"[Early Access, Action, Adventure, Indie]",http://steamcommunity.com/app/708070/reviews/?...,"[Single-player, Steam Achievements]",9.854708,1.0,708070.0,Rechargecomplete
120407,Lunar giant studios,"[Indie, Action, Co-op]",Tetrapulse,Tetrapulse,http://store.steampowered.com/app/250440/Tetra...,NaT,"[Indie, Action, Co-op]",http://steamcommunity.com/app/250440/reviews/?...,"[Multi-player, Co-op, Shared/Split Screen, Ste...",8.833768,0.0,250440.0,Lunar giant studios
120433,Versovr,"[Early Access, Indie, Casual, Simulation, Sports]",Cricket club,Cricket Club,http://store.steampowered.com/app/772180/Crick...,NaT,"[Early Access, Indie, Casual, Simulation, Sports]",http://steamcommunity.com/app/772180/reviews/?...,[Single-player],9.854708,1.0,772180.0,Versovr


In [46]:
#Creamos una nueva columna llamada 'release_date_unix' en steam_games y le asignamos valores numéricos correspondientes 
#a la marca de tiempo UNIX de la columna existente 'release_date', para luego trabajar en dicha columna

steam_games['release_date_unix'] = steam_games['release_date'].apply(lambda x: int(x.timestamp()) if not pd.isna(x) else None)

C:\Users\belen\AppData\Local\Temp\ipykernel_720\232911553.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['release_date_unix'] = steam_games['release_date'].apply(lambda x: int(x.timestamp()) if not pd.isna(x) else None)


In [47]:
#Imputamos valores nulos de la columna release_date_unix

from sklearn.impute import KNNImputer 

imputador_knn = KNNImputer(n_neighbors=5)
steam_games[['release_date_unix','price', 'early_access']] = imputador_knn.fit_transform(steam_games[['release_date_unix','price', 'early_access']])


C:\Users\belen\AppData\Local\Temp\ipykernel_720\642508167.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games[['release_date_unix','price', 'early_access']] = imputador_knn.fit_transform(steam_games[['release_date_unix','price', 'early_access']])


In [48]:
steam_games.describe()


,release_date,price,early_access,id,release_date_unix
count,29773,32124.000000,32124.000000,3.212400e+04,3.212400e+04
mean,2015-04-21 07:58:00.052396288,8.895646,0.060609,4.517508e+05,1.432325e+09
min,1970-07-15 00:00:00,0.000000,0.000000,1.000000e+01,1.684800e+07
25%,2014-10-01 00:00:00,2.990000,0.000000,3.292675e+05,1.415750e+09
50%,2016-04-18 00:00:00,4.990000,0.000000,4.520300e+05,1.464912e+09
75%,2017-04-17 00:00:00,9.990000,0.000000,5.934125e+05,1.490227e+09
max,2021-12-31 00:00:00,995.000000,1.000000,2.028850e+06,1.640909e+09
std,NaN,15.986925,0.238615,1.827313e+05,1.078922e+08


In [49]:
#Rellenamos los valores nulos de la columna 'release_date' con los valores imputados de la columna 'release_date_unix'

steam_games['release_date'] = steam_games['release_date'].fillna(steam_games['release_date_unix'])

C:\Users\belen\AppData\Local\Temp\ipykernel_720\3412151891.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['release_date'] = steam_games['release_date'].fillna(steam_games['release_date_unix'])


In [50]:
steam_games['release_date']

88310     2018-01-04 00:00:00
88311     2018-01-04 00:00:00
88312     2017-07-24 00:00:00
88313     2017-12-07 00:00:00
88314            1479168000.0
                 ...         
120440    2018-01-04 00:00:00
120441    2018-01-04 00:00:00
120442    2018-01-04 00:00:00
120443    2017-09-02 00:00:00
120444           1478528640.0
Name: release_date, Length: 32124, dtype: object

In [51]:
#Pasamos a formato Datetime, los valores de la columna 'release_date' que sean de tipo Unix

steam_games['release_date'] = pd.to_datetime(steam_games['release_date_unix'], unit='s')


C:\Users\belen\AppData\Local\Temp\ipykernel_720\3991488098.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['release_date'] = pd.to_datetime(steam_games['release_date_unix'], unit='s')


In [52]:
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_date_unix
88310,Kotoshiro,"[Strategy, Action, Indie, Casual, Simulation]",Lost summoner kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04 00:00:00,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,1.515024e+09
88311,"Making fun, inc.","[Free to Play, Strategy, Indie, RPG, Card Game...",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04 00:00:00,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,"Making fun, inc.",1.515024e+09
88312,Poolians.com,"[Free to Play, Simulation, Sports, Casual, Ind...",Real pool 3d - poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24 00:00:00,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com,1.500854e+09
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07 00:00:00,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域,1.512605e+09
88314,Otros,"[Action, Indie, Casual, Sports]",Log challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,2016-11-15 00:00:00,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,Otros,1.479168e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_rus games,"[Strategy, Indie, Casual, Simulation]",Colony on mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04 00:00:00,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,Ghost_rus games,1.515024e+09
120441,Sacada,"[Strategy, Indie, Casual]",Logistical: south africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04 00:00:00,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada,1.515024e+09
120442,Laush studio,"[Indie, Simulation, Racing]",Russian roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04 00:00:00,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush studio,1.515024e+09
120443,Sixnails,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",Exit 2 - directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02 00:00:00,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,Sixnails,1.504310e+09


In [53]:
steam_games.isna().sum()

publisher              0
genres               131
app_name               0
title                  0
url                    0
release_date           0
tags                 131
reviews_url            0
specs                662
price                  0
early_access           0
id                     0
developer              0
release_date_unix      0
dtype: int64

In [54]:
#Se eliminan los valores nulos restantes de las columnas tags, genres y specs al representar un pequeño porcentaje
#del total del dataframe 

steam_games.dropna(subset=['tags'], inplace=True)

steam_games.dropna(subset=['genres'], inplace=True)

steam_games.dropna(subset=['specs'], inplace=True)

C:\Users\belen\AppData\Local\Temp\ipykernel_720\4057114975.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games.dropna(subset=['tags'], inplace=True)
C:\Users\belen\AppData\Local\Temp\ipykernel_720\4057114975.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games.dropna(subset=['genres'], inplace=True)
C:\Users\belen\AppData\Local\Temp\ipykernel_720\4057114975.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stea

In [55]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31331 entries, 88310 to 120444
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   publisher          31331 non-null  object        
 1   genres             31331 non-null  object        
 2   app_name           31331 non-null  object        
 3   title              31331 non-null  object        
 4   url                31331 non-null  object        
 5   release_date       31331 non-null  datetime64[ns]
 6   tags               31331 non-null  object        
 7   reviews_url        31331 non-null  object        
 8   specs              31331 non-null  object        
 9   price              31331 non-null  float64       
 10  early_access       31331 non-null  float64       
 11  id                 31331 non-null  float64       
 12  developer          31331 non-null  object        
 13  release_date_unix  31331 non-null  float64       
dtypes: dat

In [56]:
#Creamos la columna 'año' para quedarnos solamente con ese valor de la columna 'release_date'

steam_games['año'] = steam_games['release_date'].dt.year

C:\Users\belen\AppData\Local\Temp\ipykernel_720\2797077276.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games['año'] = steam_games['release_date'].dt.year


In [57]:
steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_date_unix,año
88310,Kotoshiro,"[Strategy, Action, Indie, Casual, Simulation]",Lost summoner kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04 00:00:00,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,1.515024e+09,2018
88311,"Making fun, inc.","[Free to Play, Strategy, Indie, RPG, Card Game...",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04 00:00:00,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,"Making fun, inc.",1.515024e+09,2018
88312,Poolians.com,"[Free to Play, Simulation, Sports, Casual, Ind...",Real pool 3d - poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24 00:00:00,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com,1.500854e+09,2017
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07 00:00:00,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域,1.512605e+09,2017
88314,Otros,"[Action, Indie, Casual, Sports]",Log challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,2016-11-15 00:00:00,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,Otros,1.479168e+09,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_rus games,"[Strategy, Indie, Casual, Simulation]",Colony on mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04 00:00:00,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,Ghost_rus games,1.515024e+09,2018
120441,Sacada,"[Strategy, Indie, Casual]",Logistical: south africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04 00:00:00,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada,1.515024e+09,2018
120442,Laush studio,"[Indie, Simulation, Racing]",Russian roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04 00:00:00,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush studio,1.515024e+09,2018
120443,Sixnails,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",Exit 2 - directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02 00:00:00,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,Sixnails,1.504310e+09,2017


In [58]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31331 entries, 88310 to 120444
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   publisher          31331 non-null  object        
 1   genres             31331 non-null  object        
 2   app_name           31331 non-null  object        
 3   title              31331 non-null  object        
 4   url                31331 non-null  object        
 5   release_date       31331 non-null  datetime64[ns]
 6   tags               31331 non-null  object        
 7   reviews_url        31331 non-null  object        
 8   specs              31331 non-null  object        
 9   price              31331 non-null  float64       
 10  early_access       31331 non-null  float64       
 11  id                 31331 non-null  float64       
 12  developer          31331 non-null  object        
 13  release_date_unix  31331 non-null  float64       
 14  año   

In [59]:
# Una vez que tenemos el DF limpio, eliminamos las columnas que no se van a utilizar para el EDA

columnas_a_eliminar = ['title', 'url', 'release_date', 'tags', 'reviews_url','publisher', 'early_access', 'release_date_unix']

# Eliminar las columnas
steam_games = steam_games.drop(columnas_a_eliminar, axis=1)

In [60]:
steam_games

,genres,app_name,specs,price,id,developer,año
88310,"[Strategy, Action, Indie, Casual, Simulation]",Lost summoner kitty,[Single-player],4.99,761140.0,Kotoshiro,2018
88311,"[Free to Play, Strategy, Indie, RPG, Card Game...",Ironbound,"[Single-player, Multi-player, Online Multi-Pla...",0.00,643980.0,"Making fun, inc.",2018
88312,"[Free to Play, Simulation, Sports, Casual, Ind...",Real pool 3d - poolians,"[Single-player, Multi-player, Online Multi-Pla...",0.00,670290.0,Poolians.com,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,[Single-player],0.99,767400.0,彼岸领域,2017
88314,"[Action, Indie, Casual, Sports]",Log challenge,"[Single-player, Full controller support, HTC V...",2.99,773570.0,Otros,2016
...,...,...,...,...,...,...,...
120440,"[Strategy, Indie, Casual, Simulation]",Colony on mars,"[Single-player, Steam Achievements]",1.99,773640.0,Ghost_rus games,2018
120441,"[Strategy, Indie, Casual]",Logistical: south africa,"[Single-player, Steam Achievements, Steam Clou...",4.99,733530.0,Sacada,2018
120442,"[Indie, Simulation, Racing]",Russian roads,"[Single-player, Steam Achievements, Steam Trad...",1.99,610660.0,Laush studio,2018
120443,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",Exit 2 - directions,"[Single-player, Steam Achievements, Steam Cloud]",4.99,658870.0,Sixnails,2017


In [61]:
# Convierte las listas en la columna 'genres' en una cadena separada por comas
steam_games['genres'] = steam_games['genres'].apply(lambda x: str(x).replace('[', '').replace(']', ''))


# Separa la cadena por comas y agrega una fila por cada valor individual
steam_games['genres'] = steam_games['genres'].str.split(', ')
steam_games = steam_games.explode('genres')
steam_games['genres'] = steam_games['genres'].apply(lambda x: str(x).replace("'", ""))

In [62]:
# Cuenta la frecuencia de cada género
frecuencia_genero = steam_games['genres'].value_counts()

In [63]:
steam_games

,genres,app_name,specs,price,id,developer,año
88310,Strategy,Lost summoner kitty,[Single-player],4.99,761140.0,Kotoshiro,2018
88310,Action,Lost summoner kitty,[Single-player],4.99,761140.0,Kotoshiro,2018
88310,Indie,Lost summoner kitty,[Single-player],4.99,761140.0,Kotoshiro,2018
88310,Casual,Lost summoner kitty,[Single-player],4.99,761140.0,Kotoshiro,2018
88310,Simulation,Lost summoner kitty,[Single-player],4.99,761140.0,Kotoshiro,2018
...,...,...,...,...,...,...,...
120444,Adventure,Maze run vr,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,681550.0,Otros,2016
120444,Indie,Maze run vr,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,681550.0,Otros,2016
120444,Action,Maze run vr,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,681550.0,Otros,2016
120444,Simulation,Maze run vr,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,681550.0,Otros,2016


In [64]:
# Convierte las listas en la columna 'specs' en una cadena separada por comas
steam_games['specs'] = steam_games['specs'].apply(lambda x: str(x).replace('[', '').replace(']', ''))


# Separa la cadena por comas y agrega una fila por cada valor individual
steam_games['specs'] = steam_games['specs'].str.split(', ')
steam_games = steam_games.explode('specs')
steam_games['specs'] = steam_games['specs'].apply(lambda x: str(x).replace("'", ""))

In [65]:
# Cuenta la frecuencia de cada género
frecuencia_genero = steam_games['specs'].value_counts()

In [66]:
steam_games

,genres,app_name,specs,price,id,developer,año
88310,Strategy,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
88310,Action,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
88310,Indie,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
88310,Casual,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
88310,Simulation,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
...,...,...,...,...,...,...,...
120444,VR,Maze run vr,HTC Vive,4.99,681550.0,Otros,2016
120444,VR,Maze run vr,Oculus Rift,4.99,681550.0,Otros,2016
120444,VR,Maze run vr,Tracked Motion Controllers,4.99,681550.0,Otros,2016
120444,VR,Maze run vr,Standing,4.99,681550.0,Otros,2016


In [62]:
# Se crea un nuevo DF solamente con las columnas que se van a utilizar de steam_games para las funciones solicitadas

columnas_sg = ['app_name', 'genres', 'año', 'id' ]


nuevo_sg = steam_games[columnas_sg].copy()

In [63]:
nuevo_sg

,app_name,genres,año,id
88310,Lost summoner kitty,"[Strategy, Action, Indie, Casual, Simulation]",2018,761140.0
88311,Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...",2018,643980.0
88312,Real pool 3d - poolians,"[Free to Play, Simulation, Sports, Casual, Ind...",2017,670290.0
88313,弹炸人2222,"[Action, Adventure, Casual]",2017,767400.0
88314,Log challenge,"[Action, Indie, Casual, Sports]",2016,773570.0
...,...,...,...,...
120440,Colony on mars,"[Strategy, Indie, Casual, Simulation]",2018,773640.0
120441,Logistical: south africa,"[Strategy, Indie, Casual]",2018,733530.0
120442,Russian roads,"[Indie, Simulation, Racing]",2018,610660.0
120443,Exit 2 - directions,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",2017,658870.0


In [64]:
nuevo_sg.to_csv('../Data/steam_games_limpio.csv',index=False)

In [65]:
with gzip.open('../Data/steam_games_limpio.csv.gz', 'wb') as f:
    nuevo_sg.to_csv(f, index=False, encoding='utf-8')

In [67]:
steam_games.to_csv('../EDA/Data-EDA/steam_games_EDA.csv',index=False)

In [68]:
with gzip.open('../EDA/Data-EDA/steam_games_EDA.csv.gz', 'wb') as f:
    steam_games.to_csv(f, index=False, encoding='utf-8')